# Exploratory

## Notes

* **FGTS:** Refere-se a débitos de empregadores com o fundo de garantia.
* **PREV (Previdenciário):** Débitos de contribuições previdenciárias.
* **SIDA:** Conjunto de dados do sistema unificado da administração federal, com dados detalhados sobre devedores e sua situação perante a União.

Layout dos arquivos:

➜ FGTS PREV e SIDA
| Campo                     | Descrição                                                                 |
|---------------------------|---------------------------------------------------------------------------|
| CPF_CNPJ                  | CPF ou CNPJ do devedor inscrito na dívida ativa.                         |
| TIPO_PESSOA               | Indica se o devedor é pessoa física (F) ou jurídica (J).                 |
| TIPO_DEVEDOR              | Categoria do devedor: contribuinte individual, empresa, órgão público etc. |
| NOME_DEVEDOR              | Nome ou razão social do devedor.                                         |
| UF_DEVEDOR                | Unidade da Federação (estado) do devedor.                               |
| UNIDADE_RESPONSAVEL       | Unidade da PGFN responsável pela inscrição da dívida.                   |
| NUMERO_INSCRICAO          | Número único da inscrição da dívida ativa.                              |
| TIPO_SITUACAO_INSCRICAO   | Tipo da situação da inscrição: ativa, suspensa, extinta etc.            |
| SITUACAO_INSCRICAO        | Descrição detalhada da situação atual da dívida.                        |
| TIPO_CREDITO              | Tipo de crédito tributário ou não tributário (ex: FGTS, Multas).        |
| DATA_INSCRICAO            | Data da inscrição na dívida ativa.                                      |
| INDICADOR_AJUIZADO        | Indica se a dívida foi ajuizada: 'S' ou 'N'.                            |
| VALOR_CONSOLIDADO         | Valor total da dívida com acréscimos legais.                            |

➜ FGTS
| Campo                  | Descrição                                           |
|------------------------|---------------------------------------------------|
| ENTIDADE_RESPONSAVEL   | Entidade que gerou a inscrição do débito FGTS.    |
| UNIDADE_INSCRICAO      | Unidade/agência que realizou a inscrição do débito FGTS. |


## Init

### Functions

In [1]:
import pandas as pd
import zipfile
from io import TextIOWrapper

def read_zip_csvs_as_df(zip_path, encoding="utf-8", sep=";", verbose=True):
    """
    Reads a .zip file, extracts all CSVs, and concatenates them into a single DataFrame.

    :param zip_path: Path to the .zip file.
    :param encoding: Encoding used in the CSV files.
    :param sep: Field separator used in the CSV files.
    :param verbose: If True, prints file processing status.
    :return: A pandas DataFrame containing the concatenated CSV data.
    """
    dataframes = []

    with zipfile.ZipFile(zip_path, 'r') as z:
        for file_name in z.namelist():
            if file_name.endswith(".csv"):
                if verbose:
                    print(f"📂 Reading: {file_name}")
                with z.open(file_name) as f:
                    df = pd.read_csv(TextIOWrapper(f, encoding=encoding), sep=sep)
                    df["__source_file__"] = file_name  # Optional: tag for source file
                    dataframes.append(df)

    if not dataframes:
        raise ValueError("No CSV files found in the ZIP archive.")

    combined_df = pd.concat(dataframes, ignore_index=True)
    return combined_df


In [ ]:

read_zip_csvs_as_df("")